# Usage Examples

## Get Conversion Code

**When metadata as input**

In [3]:
from src.imgdataconvertcodegen import (get_conversion, get_convert_path,
                                       add_image_metadata, add_convert_code_factory,
                                       add_new_lib_preset)

source_metadata = {
    "data_representation": "numpy.ndarray",
    "color_channel": "rgb",
    "channel_order": "channel last",
    "minibatch_input": False,
    "data_type": "uint8",
    "intensity_range": "full",
    "device": "cpu"
}
target_metadata = {
    "data_representation": "torch.tensor",
    "color_channel": "rgb",
    "channel_order": "channel first",
    "minibatch_input": True,
    "data_type": "uint8",
    "intensity_range": "full",
    "device": "cpu"
}

conversion = get_conversion("input", source_metadata, "output", target_metadata)

print(f"The imports code is: \n{conversion[0]}\n")
print(f"The conversion code is: \n{conversion[1]}")

The imports code is: 
import torch

The conversion code is: 
var_af065e5b09c04bbeaf334306e17b3f79 = torch.from_numpy(input)
var_fd0c641b704b4b748c58fbe8e281b6a8 = var_af065e5b09c04bbeaf334306e17b3f79.permute(2, 0, 1)
output = torch.unsqueeze(var_fd0c641b704b4b748c58fbe8e281b6a8, 0)


**When library name as input**

In [6]:
code_using_lib_name = get_conversion("input", 'scikit-image', "output", 'torch', 'color', 'color')

print(f"The imports code is: \n{conversion[0]}\n")
print(f"The conversion code is: \n{conversion[1]}")

The imports code is: 
import torch

The conversion code is: 
var_af065e5b09c04bbeaf334306e17b3f79 = torch.from_numpy(input)
var_fd0c641b704b4b748c58fbe8e281b6a8 = var_af065e5b09c04bbeaf334306e17b3f79.permute(2, 0, 1)
output = torch.unsqueeze(var_fd0c641b704b4b748c58fbe8e281b6a8, 0)


## Get Conversion Path

In [7]:
path = get_convert_path(source_metadata, target_metadata)

print("the path is:")
for metadata in path:
    print(f'\t{metadata}')

the path is:
	{'data_representation': 'numpy.ndarray', 'color_channel': 'rgb', 'channel_order': 'channel last', 'minibatch_input': False, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}
	{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel last', 'minibatch_input': False, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}
	{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel first', 'minibatch_input': False, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}
	{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel first', 'minibatch_input': True, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}


## Convert Image Data

In [8]:
import numpy as np
import torch

source_image = np.random.randint(0, 256, (20, 20, 3), dtype=np.uint8)
expected_image = torch.from_numpy(source_image).unsqueeze(0).permute(0, 3, 1, 2)
code = get_conversion("source_image", source_metadata, "actual_image", target_metadata)
exec(code[1])

## Add new Metadata spec

In [12]:
add_image_metadata({
    "data_representation": "PIL.Image",
    "color_channel": "rgb",
    "channel_order": "channel last",
    "minibatch_input": False,
    "data_type": "uint8",
    "intensity_range": "0to255",
    "device": "cpu"
})

## Add a new factory

In [11]:
def convert_torch_to_numpy(source, target):
    if source["data_representation"] == "torch.tensor" and target["data_representation"] == "numpy.ndarray":
        return '', f"actual_image = source_image.numpy()"
    return None


add_convert_code_factory(convert_torch_to_numpy)

## Add a new librariy metadata preset

In [10]:
add_new_lib_preset("PIL.image", 'color', {
    "data_representation": "PIL.Image",
    "color_channel": "rgb",
    "channel_order": "channel last",
    "minibatch_input": False,
    "data_type": "uint8",
    "intensity_range": "full",
    "device": "cpu"
})